# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_weather_data = pd.read_csv('city_weather_data.csv').drop(['Unnamed: 0'], axis=1)
city_weather_data


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [ ]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_data[["Lat", "Lng"]].astype(float)

# Store humidty as weight
humidity = city_weather_data["Humidity"].astype(float)

In [101]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 5)

fig.add_layer(heat_layer)

fig

ValueError: weights must be of the same length as locations or None

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#A max temperature lower than 80 degrees but higher than 20.
#Wind speed less than 10 mph.
#Less than 10% cloudiness

hotel_df = city_weather_data.loc[(city_weather_data["Max Temp"] > 20) & (city_weather_data["Max Temp"] < 80) & (city_weather_data["Wind Speed"] < 10) & (city_weather_data["Cloudiness"] < 10) ]
hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# Fixed params independent of each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Use the lat/lng 
for index, row in hotel_df.iterrows():
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    if row["Hotel Name"]=="":

        # change location params
        params["location"] = f"{lat},{lng}"

        # make request 
        name = requests.get(base_url, params=params).json()
        

        hotel_df.loc[index, "Hotel Name"] = name["results"][0]["name"]

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations,info_box_content= hotel_info)
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

